# Stockfish Highest ELO vs. Stockfish 700 ELO

In [1]:
import chess
from stockfish import Stockfish
from ollama import chat
from ollama import ChatResponse
import json
import random
import pandas as pd
import asyncio

In [2]:
HIGH_ELO = 3500
LOW_ELO = 1000
NUM_GAMES = 100
AI_USE_US = 80
AI_USE_CN = 80

In [3]:
stockfish_us = Stockfish()
stockfish_cn = Stockfish()

In [4]:
cn_wins = 0
us_wins = 0
draws = 0

In [5]:
async def get_move_with_timeout(stockfish, timeout=15):
    """
    Run stockfish.get_best_move() in a separate thread and wait for a maximum of `timeout` seconds.
    If the call exceeds the timeout, return None.
    """
    try:
        # Use asyncio.to_thread to run the blocking call in a separate thread.
        move = await asyncio.wait_for(asyncio.to_thread(stockfish.get_best_move), timeout)
        return move
    except asyncio.TimeoutError:
        # move = stockfish.get_best_move_time(1000)
        print(f"Timeout: stockfish.get_best_move() took more than 15 seconds.")
        return None

In [6]:
for game_number in range(1, NUM_GAMES + 1):
    
    move_counter = 0
    num_us = random.randint(1, 100)
    num_cn = random.randint(1, 100)

    if num_us <= AI_USE_US: stockfish_us.update_engine_parameters({"Skill Level": 20, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})
    else: stockfish_us.update_engine_parameters({"Skill Level": 10, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})

    if num_cn <= AI_USE_CN: stockfish_cn.update_engine_parameters({"Skill Level": 20, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})
    else: stockfish_cn.update_engine_parameters({"Skill Level": 10, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})

    print(stockfish_us.get_parameters())
    print(stockfish_cn.get_parameters())

    print(f"Starting game {game_number}...")

    board = chess.Board()

    while not board.is_game_over():
        # Determine which engine to use based on turn
        current_engine = stockfish_us if board.turn == chess.WHITE else stockfish_cn

        # Set the position on the board for the engine
        current_engine.set_fen_position(board.fen())

        # Get the best move from the engine
        best_move = await get_move_with_timeout(current_engine)
        if best_move is None:
            print("Engine failed to provide a move.")
            # if current_engine == stockfish_us:
            #     stockfish_us = Stockfish(parameters={})
            break

        # Apply the move to the board
        print(f"Move by {"US White" if board.turn else "CN Black"}: {best_move}   Move: {move_counter}")
        move = chess.Move.from_uci(best_move)
        board.push(move)
        move_counter += 1

    # Record the result of the game
    result = board.result()
    if result == "1-0":
        us_wins += 1
    elif result == "0-1":
        cn_wins += 1
    else:
        draws += 1

    print(f"Game {game_number} result: {result}\n\n")

{'Debug Log File': '', 'Contempt': 0, 'Min Split Depth': 0, 'Ponder': 'false', 'MultiPV': 1, 'Skill Level': 20, 'Move Overhead': 10, 'Minimum Thinking Time': 50, 'Slow Mover': 100, 'UCI_Chess960': 'false', 'UCI_LimitStrength': 'false', 'UCI_Elo': 1350, 'Threads': 3, 'Hash': 2048}
{'Debug Log File': '', 'Contempt': 0, 'Min Split Depth': 0, 'Ponder': 'false', 'MultiPV': 1, 'Skill Level': 20, 'Move Overhead': 10, 'Minimum Thinking Time': 50, 'Slow Mover': 100, 'UCI_Chess960': 'false', 'UCI_LimitStrength': 'false', 'UCI_Elo': 1350, 'Threads': 3, 'Hash': 2048}
Starting game 1...
Move by US White: e2e4   Move: 0
Move by CN Black: c7c5   Move: 1
Move by US White: g1f3   Move: 2
Move by CN Black: b8c6   Move: 3
Move by US White: f1b5   Move: 4
Move by CN Black: g7g6   Move: 5
Move by US White: b5c6   Move: 6
Move by CN Black: b7c6   Move: 7
Move by US White: e1g1   Move: 8
Move by CN Black: f8g7   Move: 9
Move by US White: f1e1   Move: 10
Move by CN Black: d8c7   Move: 11
Move by US White: c2c

In [7]:
results = []

In [8]:
results.append({
            "US Wins": us_wins,
            "CN Wins": cn_wins,
            "Draws": draws,
        })

print(results)

[{'US Wins': 25, 'CN Wins': 16, 'Draws': 59}]


## Swapping sides

In [11]:
stockfish_us = Stockfish()
stockfish_cn = Stockfish()

In [12]:
us_wins = 25
cn_wins = 16
draws = 59

In [13]:
for game_number in range(1, NUM_GAMES + 1):
    
    move_counter = 0
    num_us = random.randint(1, 100)
    num_cn = random.randint(1, 100)

    if num_us <= AI_USE_US: stockfish_us.update_engine_parameters({"Skill Level": 20, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})
    else: stockfish_us.update_engine_parameters({"Skill Level": 10, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})

    if num_cn <= AI_USE_CN: stockfish_cn.update_engine_parameters({"Skill Level": 20, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})
    else: stockfish_cn.update_engine_parameters({"Skill Level": 10, "Threads": 3, "Hash": 2048, "Minimum Thinking Time": 50})

    print(stockfish_us.get_parameters())
    print(stockfish_cn.get_parameters())

    print(f"Starting game {game_number}...")

    board = chess.Board()

    while not board.is_game_over():
        # Determine which engine to use based on turn
        current_engine = stockfish_cn if board.turn == chess.WHITE else stockfish_us

        # Set the position on the board for the engine
        current_engine.set_fen_position(board.fen())

        # Get the best move from the engine
        best_move = await get_move_with_timeout(current_engine)
        if best_move is None:
            print("Engine failed to provide a move.")
            # if current_engine == stockfish_us:
            #     stockfish_us = Stockfish(parameters={})
            break

        # Apply the move to the board
        print(f"Move by {"CN White" if board.turn else "US Black"}: {best_move}   Move: {move_counter}")
        move = chess.Move.from_uci(best_move)
        board.push(move)
        move_counter += 1

    # Record the result of the game
    result = board.result()
    if result == "1-0":
        cn_wins += 1
    elif result == "0-1":
        us_wins += 1
    else:
        draws += 1

    print(f"Game {game_number} result: {result}\n\n")

{'Debug Log File': '', 'Contempt': 0, 'Min Split Depth': 0, 'Ponder': 'false', 'MultiPV': 1, 'Skill Level': 20, 'Move Overhead': 10, 'Minimum Thinking Time': 50, 'Slow Mover': 100, 'UCI_Chess960': 'false', 'UCI_LimitStrength': 'false', 'UCI_Elo': 1350, 'Threads': 3, 'Hash': 2048}
{'Debug Log File': '', 'Contempt': 0, 'Min Split Depth': 0, 'Ponder': 'false', 'MultiPV': 1, 'Skill Level': 20, 'Move Overhead': 10, 'Minimum Thinking Time': 50, 'Slow Mover': 100, 'UCI_Chess960': 'false', 'UCI_LimitStrength': 'false', 'UCI_Elo': 1350, 'Threads': 3, 'Hash': 2048}
Starting game 1...
Move by CN White: e2e4   Move: 0
Move by US Black: e7e5   Move: 1
Move by CN White: g1f3   Move: 2
Move by US Black: b8c6   Move: 3
Move by CN White: f1b5   Move: 4
Move by US Black: a7a6   Move: 5
Move by CN White: b5a4   Move: 6
Move by US Black: g8f6   Move: 7
Move by CN White: e1g1   Move: 8
Move by US Black: f8e7   Move: 9
Move by CN White: f1e1   Move: 10
Move by US Black: b7b5   Move: 11
Move by CN White: a4b

In [14]:
results = []

In [15]:
results.append({
            "US Wins": us_wins,
            "CN Wins": cn_wins,
            "Draws": draws,
        })

print(results)

[{'US Wins': 40, 'CN Wins': 46, 'Draws': 114}]


In [16]:
df = pd.DataFrame(results)
csv_path = f"Results/STKFISH_{AI_USE_US}_vs_STKFISH_{AI_USE_CN}.csv"
df.to_csv(csv_path, index=False)
print(f"Results saved to {csv_path}")

Results saved to Results/STKFISH_80_vs_STKFISH_80.csv
